In [1]:
import numpy as np
import tensorflow as tf
import cv2 as cv
import matplotlib.pyplot as plt
import os

In [2]:
def OD_testing(model_path, testing_image_path, output_dir_path):
    """
    model_path: enter the model path ie. F:/model/tensor.pb
    
    testing_image_path: enter the directory path that contain only images 
    
    output_dir_path: write the tested images 
    """
    
    
    # Read the graph.
    print("model dir path >>>",model_path)
    with tf.gfile.FastGFile(model_path, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    with tf.Session() as sess:
      # Restore session
        sess.graph.as_default()
        tf.import_graph_def(graph_def, name='')


        # Read and preprocess an image.
        print("testing image dir path >>>",testing_image_path)
        print("output image dir path >>>",output_dir_path)
        for i in os.listdir(testing_image_path):
            image_path = testing_image_path+"/"+i

            img = cv.imread(image_path)
            rows = img.shape[0]
            cols = img.shape[1]
            inp = cv.resize(img, (500, 500))
            inp = inp[:, :, [2, 1, 0]] # BGR2RGB

            out = sess.run([sess.graph.get_tensor_by_name('num_detections:0'),
            sess.graph.get_tensor_by_name('detection_scores:0'),
            sess.graph.get_tensor_by_name('detection_boxes:0'),
            sess.graph.get_tensor_by_name('detection_classes:0')],
            feed_dict={'image_tensor:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})

            # Visualize detected bounding boxes.
            num_detections = int(out[0][0])
            for j in range(num_detections):
                classId = int(out[3][0][j])
                score = float(out[1][0][j])
                bbox = [float(v) for v in out[2][0][j]]
                if score > 0.4:
                    x = bbox[1] * cols
                    y = bbox[0] * rows
                    right = bbox[3] * cols
                    bottom = bbox[2] * rows
                    cv.rectangle(img, (int(x), int(y)), (int(right), int(bottom)), (0, 0, 255), thickness=2)

            cv.imwrite(output_dir_path+"/"+i,img)
    #         cv.imshow('TensorFlow MobileNet-SSD', img)
            cv.waitKey()
            cv.destroyAllWindows()

In [3]:
model = r"D:/Projects/Dairy_machine_ocr/frozen_inference_graph (1) (1).pb" # model file created form object training
path = r"D:\Projects\Dairy_machine_ocr\dataset-v1\test" #Test data path
output = r"D:\Projects\Dairy_machine_ocr\dataset-v1\test_output" # data output path

In [4]:
OD_testing(model, path, output)

model dir path >>> D:/Projects/Dairy_machine_ocr/frozen_inference_graph (1) (1).pb
Instructions for updating:
Use tf.gfile.GFile.
testing image dir path >>> D:\Projects\Dairy_machine_ocr\dataset-v1\test
output image dir path >>> D:\Projects\Dairy_machine_ocr\dataset-v1\test_output
